classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
from config import *

import json
import copy
import math

from torch.distributions.multivariate_normal import MultivariateNormal


In [2]:
torch.cuda.is_available()


True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=1, image_transforms=None)

In [21]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=5, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [5]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 15
dataset = 'miniimagenet'
root = '../../dataset/mini_imagenet'
order = False
inner_lr = 0.01
meta_lr = 0.001
reconst_loss = nn.MSELoss(reduction='mean')
inner_adapt_steps_train = 5

In [6]:
train_tasks, valid_tasks, test_tasks, learner = setup(
    dataset, root, n_ways, k_shots, q_shots, order, inner_lr, device, download='False')
opt = optim.Adam(learner.parameters(), meta_lr)

In [7]:
opt.zero_grad()
meta_train_loss = []
meta_valid_loss = []
meta_train_acc = []
meta_valid_acc = []


In [8]:
ttask = train_tasks.sample()
model = learner.clone()
evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
    ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device)


/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
evaluation_loss

(tensor(6.3777, grad_fn=<AddBackward0>),
 tensor(1.6141, grad_fn=<NllLossBackward>),
 tensor(0.1253, grad_fn=<MseLossBackward>),
 tensor(2.1628, grad_fn=<MeanBackward0>),
 tensor(2.4754, grad_fn=<MeanBackward0>))

In [10]:
evaluation_loss[0].backward()
meta_train_loss.append(evaluation_loss[0])
meta_train_acc.append(evaluation_accuracy)


In [11]:
evaluation_loss[0]

tensor(16.9887, grad_fn=<AddBackward0>)

In [ ]:
for p in learner.parameters():
    p.grad.data.mul_(1.0 / meta_batch_size)
opt.step()

In [ ]:
data, labels = ttask
data, labels = data.to(device), labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]

# Inner adapt step
for _ in range(1):
    reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(support)
    adapt_loss = loss(reconst_loss, reconst_image, support/256, logits, support_labels, mu_s, log_var_s, mu_l, log_var_l)
    learner.adapt(adapt_loss[0])

reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = learner(queries)
eval_loss = loss(reconst_loss, reconst_image, queries/256, logits, queries_labels, mu_s, log_var_s, mu_l, log_var_l)
eval_acc = accuracy(F.softmax(logits, dim=1), queries_labels)